In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import utils
line ="if (x == 2.2 or (y > 3 and z < 2):)"
l, s, c, uc = utils.segmentLine(line)
print(l)
print(s)
print(uc)

['if', '(', 'x', '==', 'NUM', '.', 'NUM', 'or', '(', 'y', '>', 'NUM', 'and', 'z', '<', 'NUM', ')', ')']
{'', 'z', '<', 'y', '>', '==', 'if', 'x', 'and', 'NUM', 'or'}
{'if': [1], '(': [4, 17], 'x': [5], '==': [7], 'NUM': [10, 12, 22, 32], '.': [11], 'or': [14], 'y': [18], '>': [20], 'and': [24], 'z': [28], '<': [30], ')': [33, 35]}


In [11]:
import ast
from operators import *
import operators
import unparseAST
import utils
import time

def build_name_to_operator_map():
    result = {}
    for operator in operators.standard_operators | operators.experimental_operators:
        result[operator.name()] = operator
        # result[operator.long_name()] = operator
    return result

name_operator = build_name_to_operator_map()



import time
start = time.time()
code = """s = 1 * 2;
s = 1 * 2"""

codeLines = code.split('\n')
codeLineslst = [] # temporary list to store the code lines
codeLinesset = set() # temporary set to store the code lines
print(codeLines)
for line in codeLines:
    codeLineslstX , codeLinessetX, offsets = utils.segmentLine(line) # any variable appended by X is temporary
    # codeLineslst.extend(codeLineslstX)
    # codeLinesset.update(codeLinessetX)

for line in codeLineslst:
    print(line)


lstMutations, weights = utils.mutationsCanBeApplied(codeLinesset)
print(lstMutations)


ast_node = ast.parse(code)
# print(unparseAST.to_source(ast_node))
copied = copyMutation().visit(ast_node)
# copy.deepcopy(ast_node)
mutant = (AdditionOperatorReplacement(target_node_lineno = 1, code_ast = ast_node)).visitC()
mutant = ast.fix_missing_locations(mutant) # after mutation, we need to fix the missing locations
# mutant = (BinaryOperatorReplacement(target_node_lineno= 1, code = mutant, operator='SUB')).visitC()
# print(ast.dump(mutant, indent=4))
# ast_node = copyMutation().visit(copied)
span = time.time() - start
print(span)
print(ast_node is copied)
print(unparseAST.to_source(copied))
print(unparseAST.to_source(ast_node))


# again = ArithmeticOperatorReplacement.printMutatedSet()



['s = 1 * 2;', 's = 1 * 2']
[]
0.006041049957275391
False
s = 1 * 2
s = 1 * 2
s = 1 - 2
s = 1 * 2


In [12]:
# testing removing a statement

code="""
for i in range(10):
    print(i)

if a == 1:
    print("a is 1")
else:
    print("a is not 1")

if a == 85776474:
    print("a is 85776474") 

a = 1
"""

class temp(ast.NodeTransformer):
    def visit_If(self, node):
        if node.lineno == 5:
            return None
        else:
            return node

ast_node = ast.parse(code)
m = temp().visit(ast_node)
res = unparseAST.to_source(m)
print(res)


for i in range(10):
    print(i)






if a == 85776474:
    print('a is 85776474')

a = 1


In [13]:
# # copy test
# import copy
# li1 = [1, 2, [3,5], 4]
# li2 = copy.copy(li1)
# print ("The original elements before shallow copying")
# for i in range(0,len(li1)):
# 	print (li1[i],end=" ")

# print("\r")
# li2[2][0] = 7
# li2[2][1] = 7
# li2[1] = 7
# print ("The original elements after shallow copying")
# for i in range(0,len( li1)):
# 	print (li1[i],end=" ")


In [14]:
import ast
from operators import *
import operators
import unparseAST
import utils
import time
import random

copier = copyMutation()
def getNameToOperatorMap(self):
    name_to_operator = utils.build_name_to_operator_map()
    return name_to_operator

name_to_operator = getNameToOperatorMap(operators)


import time
start = time.time()
code = """
i = 1 == 2"""



codeLines = code.split('\n')
codeLineslst = [] # temporary list to store the code lines
codeLinesset = set() # temporary set to store the code lines
print(codeLines) # print the code lines which are a list lines after being split by '\n'
for line in codeLines:
    codeLineslstX , codeLinessetX, offsets = utils.segmentLine(line) # any variable appended by X is temporary
    # assume know we are going to mutate the code line by line
    print(len(codeLineslstX), len(codeLinessetX), len(offsets))
    print(codeLineslstX)
    print(offsets)
    lstMutations, weights = utils.mutationsCanBeApplied(codeLinessetX)
    if not lstMutations: # check list is empty or not
        continue
    choice = random.choices(lstMutations, weights = weights, k=1)[0] # random.choice returns a list, its size determined by k
    i = 1
    line_ast = ast.parse(line)
    if (type(choice) is tuple):
        opCode = choice[0]
        operator = choice[1]
        op = name_to_operator[opCode]
        lineParsedOriginal = copier.visit(line_ast)
        mutant = op(target_node_lineno = i, code_ast = line_ast, operator = operator).visitC()
        mutant = ast.fix_missing_locations(mutant) # after mutation, we need to fix the missing locations
        print("**********************************")
        print(unparseAST.to_source(mutant))
        print("**********************************")
    else:
        op = name_to_operator[choice]
        lineParsedOriginal = copier.visit(line_ast)
        line_ast = ast.parse(line)
        mutant = op(target_node_lineno = i, code_ast = line_ast).visitC()
        mutant = ast.fix_missing_locations(mutant) # after mutation, we need to fix the missing locations
        print("**********************************")
        print(unparseAST.to_source(mutant))
        print("**********************************")
    # i+=1



faultyLineLocations = [1]
# for l in faultyLineLocations:
    # print(l)


lstMutations, weights = utils.mutationsCanBeApplied(codeLinesset)
# print(lstMutations)


ast_node = ast.parse(code)
# print(unparseAST.to_source(ast_node))
copied = copyMutation().visit(ast_node)
# copy.deepcopy(ast_node)
mutant = (AdditionOperatorReplacement(target_node_lineno = 1, code_ast = ast_node)).visitC()
mutant = ast.fix_missing_locations(mutant) # after mutation, we need to fix the missing locations
# mutant = (BinaryOperatorReplacement(target_node_lineno= 1, code = mutant, operator='SUB')).visitC()
# print(ast.dump(mutant, indent=4))
# ast_node = copyMutation().visit(copied)
span = time.time() - start
print(span)
# print(ast_node is copied)
# print(unparseAST.to_source(copied))
# print(unparseAST.to_source(line_ast))


# again = ArithmeticOperatorReplacement.printMutatedSet()


['', 'i = 1 == 2']
0 0 0
[]
[]
5 5 5
['i', '=', '1', '==', '2']
[1, 3, 5, 7, 10]
**********************************
i = 1 != 2
**********************************
0.002685070037841797


In [15]:
import re
from typing import List

from joblib import PrintTime
def passesNegTests(program:str, program_name:str, inputs:List, outputs:List) -> bool:
    """
    Inputs:
    program : str :  program to be tested
    inputs : List :  inputs to the program
    outputs : List :  outputs of the program
    """
    # let's try by capturing the name of the function in regex and the list of names is compared with the name of the function
    editedProgram = None
    passedTests = 0
    function_names = re.findall(r'def\s+(\w+)', program)
    for name in function_names:
        if name == program_name:
            # program = re.sub(r'$', f'\ntestcase = {inputs}', program)
            program = re.sub(r'$', f'\nres = {program_name}', program)
            # print(program)
            break

    for i in range(len(inputs)):
        try:
            testcase = inputs[i]
            program = re.sub(f'\nres = {program_name}', f'\n\ntestcase = {testcase}\nres = {program_name}', program)
            editedProgram = re.sub(f'\nres = {program_name}', f'\nres = {program_name}(*testcase)', program)
            if (globals().get('res') is not None):
                del globals()['res']
            exec(editedProgram, globals())
            print(res)
            if res == outputs[i]:
                passedTests += 1
        except Exception as e:
            print(e)
            # return False
    return passedTests

# Open the file in read mode
with open('BuggyProgram.txt', 'r') as file:
    # Read the entire content of the file
    content = file.read()
    print(content)

passedTests = passesNegTests(content, 'add', [(1, 2), (1, 2)], [(3), (2)])
print(passedTests)
PrintTime()



def add(a, b):
    return a * b
2
2
1


In [21]:
import re
from typing import List

from joblib import PrintTime
def passesNegTests(program:str, program_name:str, inputs:List, outputs:List) -> bool:
    """
    Inputs:
    program : str :  program to be tested
    inputs : List :  inputs to the program
    outputs : List :  outputs of the program
    """
    # let's try by capturing the name of the function in regex and the list of names is compared with the name of the function
    editedProgram = None
    passedTests = 0
    function_names = re.findall(r'def\s+(\w+)', program)
    for name in function_names:
        if name == program_name:
            # program = re.sub(r'$', f'\ntestcase = {inputs}', program)
            program = re.sub(r'$', f'\nres = {program_name}', program)
            # print(program)
            break

    for i in range(len(inputs)):
        try:
            testcase = inputs[i]
            program = re.sub(f'\nres = {program_name}', f'\n\ntestcase = {testcase}\nres = {program_name}', program)
            editedProgram = re.sub(f'\nres = {program_name}', f'\nres = {program_name}(*testcase)', program)
            if (globals().get('res') is not None):
                del globals()['res']
            exec(editedProgram, globals())
            print(res)
            if res == outputs[i]:
                passedTests += 1
        except Exception as e:
            print(e)
            # return False
    return passedTests

# Open the file in read mode
with open('BuggyProgram.txt', 'r') as file:
    # Read the entire content of the file
    content = file.read()
    print(content)

# passedTests = passesNegTests(content, 'add', [(1, 2), (1, 2)], [(3), (2)])
# print(passedTests)
# ast.dump(ast.parse(content))
content="""
def add(a, b):
    return a * b
"""
copier.visit(ast.parse(content))


def add(a, b):
    return a * b


AttributeError: 'NoneType' object has no attribute '_fields'

# Karim Part

In [14]:
import unparseAST
import ast
code = """
x = 2
print()
if x == 2:
    if x != 5:
        pass
    y = 2
else:
    print("x is not 2")
    y = 3
"""
code_ast = ast.parse(code)

print(ast.dump(code_ast, indent = 4))

Module(
    body=[
        Assign(
            targets=[
                Name(id='x', ctx=Store())],
            value=Constant(value=2)),
        Expr(
            value=Call(
                func=Name(id='print', ctx=Load()),
                args=[],
                keywords=[])),
        If(
            test=Compare(
                left=Name(id='x', ctx=Load()),
                ops=[
                    Eq()],
                comparators=[
                    Constant(value=2)]),
            body=[
                If(
                    test=Compare(
                        left=Name(id='x', ctx=Load()),
                        ops=[
                            NotEq()],
                        comparators=[
                            Constant(value=5)]),
                    body=[
                        Pass()],
                    orelse=[]),
                Assign(
                    targets=[
                        Name(id='y', ctx=Store())],
                    value=Cons

In [15]:
from numpy import generic
from sympy import I


class kimoMutator(ast.NodeVisitor):

    def __init__(self):
        self.parents = []
        self.indices = []
        self.target_line_no = None

    def store_parent(self, index, parent):
        self.parents.append(parent)
        self.indices.append(index)

    def set_line_no(self, linenumbers):
        self.target_line_numbers = linenumbers

    @property
    def get_indices(self):
        return self.indices

    @property
    def get_parents(self):
        return self.parents

    def visit_If(self, node):
        x = node.parent
        # new_node = ast.parse("print('hello')")
        if (node.lineno in self.target_line_numbers):
            for i, nodeParent in enumerate(node.parent.body):
                # node.parent.body
                if (nodeParent is node):
                    print("OK")
                    self.store_parent(i, node.parent)

                    # x.append(new_node)
                    # node._fields = node._fields + ("test",)
                    # setattr(node, "body", x)
                    # node.parent.body.insert(i - 1, ast.parse("print('hello')"))

            # print(node)
        return self.generic_visit(node)
    
def parentify(tree):
    tree.parent = None
    for node in ast.walk(tree):
        for child in ast.iter_child_nodes(node):
            child.parent = node

parentify(code_ast)
mutator = kimoMutator()
new_node = ast.parse("print('hello')\n")
lineNos = {4, 5}
mutator.set_line_no(lineNos)
##############################################################


mutator.visit(code_ast)
# mutator.parent.body.insert(mutator.index, new_node)
# print(mutator.parent.body)
print(unparseAST.to_source(code_ast))


OK
OK

x = 2
print()
if x == 2:
    if x != 5:
        pass
    y = 2
else:
    print('x is not 2')
    y = 3


In [16]:
print(ast.dump(new_node, indent = 4))

Module(
    body=[
        Expr(
            value=Call(
                func=Name(id='print', ctx=Load()),
                args=[
                    Constant(value='hello')],
                keywords=[]))],
    type_ignores=[])


In [17]:
for i, parent in enumerate(mutator.get_parents):
    parent.body.insert(mutator.get_indices[i], new_node.body[0])
    ast.fix_missing_locations(code_ast)
    # break
print(ast.unparse(code_ast))

x = 2
print()
print('hello')
if x == 2:
    print('hello')
    if x != 5:
        pass
    y = 2
else:
    print('x is not 2')
    y = 3
